BELOW IS A WORKING BASE VERSION: 

In [ ]:
from bokeh.io import vform
from bokeh.models import CustomJS, ColumnDataSource, Select
from bokeh.plotting import Figure, output_file, show, hplot, vplot
from bokeh.models import ColumnDataSource, HBox, VBoxForm,Range1d, HoverTool, BoxZoomTool, \
ResetTool, ResizeTool, PreviewSaveTool, PanTool, BoxSelectTool, LassoSelectTool, Paragraph, CustomJS
from bokeh.models.widgets import CheckboxGroup, Panel, Tabs
from astropy.table import Table
import numpy as np
import pandas as pd
from bokeh.models import Range1d
from bokeh.palettes import RdPu9
import matplotlib.pyplot as plt



TOOLS= [BoxZoomTool(), ResetTool(), ResizeTool(), PanTool(), HoverTool(tooltips=[("sobject_id","@sobject_id")])]
TOOLS2= [BoxZoomTool(),ResetTool(), ResizeTool(), PanTool(), HoverTool(tooltips=[("counts","@counts")])]

abund=Table.read('/Users/kschles/Documents/GALAH/wg4output/wg4_04292016/sobject_iraf_k2.fits', format='fits')
test=Table.to_pandas(abund)
abund_orig=test
test=abund_orig.loc[0:5000]
histcolumns=['sobject_id', 'ra', 'dec', 'teff_cannon', 'logg_cannon', 'feh_cannon', 'alpha_fe_cannon',\
             'alpha_fe_sme', 'teff_guess', 'logg_guess',\
             'feh_guess', 'rv_guess', 'rv_cannon','rv_sme', 'mg_abund_sme','si_abund_sme',\
             'ca_abund_sme','ti_abund_sme']

abund=test.loc[:,histcolumns]

output_file("callback.html")

axis_map = {
        "[Fe/H] Cannon": "feh_cannon",
        "[a/Fe] Cannon": "alpha_fe_cannon",
        "Teff Cannon" : "teff_cannon",
        "logg Cannon" : "logg_cannon",
        #"[Fe/H] SME" : "feh_sme",
        "[a/Fe] SME": "alpha_fe_sme",
        #"Teff SME" : "teff_sme",
        #"logg SME" : "logg_sme",
        "RA" : "ra",
        "Dec" : "dec",
        "RV Cannon": "rv_cannon",
        "RV SME" : "rv_sme",
        "RV GUESS" : "rv_guess",
        "[Fe/H] GUESS": "feh_guess",
        "Teff GUESS" : "teff_guess",
        "logg GUESS" : "logg_guess",
        "RV GUESS" : "rv_guess",
        #"Li": "li_abund_sme",
        #"C" : "c_abund_sme",
        #"O" : "o_abund_sme",
        #"Na" : "na_abund_sme",
        "Mg" : "mg_abund_sme", 
        #"Al" : "al_abund_sme", 
        "Si" : "si_abund_sme", 
        #"K"  : "k_abund_sme",
        "Ca" : "ca_abund_sme", 
        #"Sc" : "sc_abund_sme",
        "Ti" : "ti_abund_sme", 
        #"V"  : "v_abund_sme",
        #"Cr" : "cr_abund_sme", 
        #"Mn" : "mn_abund_sme",
        #"Co" : "co_abund_sme",
        #"Ni" : "ni_abund_sme",
        #"Cu" : "cu_abund_sme",
        #"Zn" : "zn_abund_sme",
        #"Y"  : "y_abund_sme",
        #"Ba" : "ba_abund_sme",
        #"La" : "la_abund_sme",
        #"Nd" : "nd_abund_sme",
        #"Eu" : "eu_abund_sme",
        #"Rb" : "rb_abund_sme"
        }


xname="[Fe/H] Cannon"
yname="[a/Fe] SME"
binval='50'

## Set up initial data: 
x_name = axis_map[xname]
y_name = axis_map[yname]

xbinnum=int(binval)
ybinnum=int(binval)


## Loop through columns in abund to add histograms to dictionary for source call in bokeh interactive.
for i in range(1,len(histcolumns)): 
    colname=histcolumns[i]
    hhist, hedges=np.histogram(abund.loc[np.where(np.isnan(abund[colname])==False)[0],colname], bins=xbinnum)
    lst=[np.nan]*(len(abund)-xbinnum) 
    lst2=[np.nan]*(len(abund)-(xbinnum+1)) 
    name1='hh_'+colname
    name2='hel_'+colname
    name3='her_'+colname
    abund[name1]=(pd.concat([pd.Series(hhist),pd.Series(lst)], ignore_index=True))
    abund[name2]=(pd.concat([pd.Series(hedges[:-1]),pd.Series(lst2)], ignore_index=True))
    abund[name3]=(pd.concat([pd.Series(hedges[1:]),pd.Series(lst2)], ignore_index=True))

## Loop through columns to add density distributions to dictionary for source call. 

histcolumns2=['sobject_id', 'ra', 'dec', 'teff_cannon', 'logg_cannon', 'feh_cannon', 'alpha_fe_cannon',\
             'alpha_fe_sme', 'teff_guess', 'logg_guess',\
             'feh_guess', 'rv_guess', 'rv_cannon','rv_sme', 'mg_abund_sme','si_abund_sme',\
             'ca_abund_sme','ti_abund_sme']

for f in range(1,len(histcolumns2)): 
    colname1=histcolumns2[f]
    for g in range(1,len(histcolumns2)): 
        colname2=histcolumns2[g]
        if (colname2!=colname1): 
            subset=abund[[colname1,colname2]].dropna()
            subset.reset_index(drop=True, inplace=True)
            if (len(subset)>50): 
                counts, xb, yb, image=plt.hist2d(subset[colname1],subset[colname2],bins=50)
                wi=xb[1]-xb[0]
                he=yb[1]-yb[0]
                colors=['#ffffff']+RdPu9[::-1]
                
                xvals=[]
                yvals=[]
                cvals=[]
                colvals=[]
                if (np.max(counts)!=np.min(counts)): 
                    counts_per_bin=(np.max(counts)-np.min(counts))/8.
                else :
                    print 'Counts Error: ', colname1,colname2
                
                for k in range(len(xb)-1): 
                    for l in range(len(xb)-1): 
                        xvals.append(xb[k])
                        yvals.append(yb[l])
                        cvals.append(counts[k][l])
                        colvals.append(colors[int(np.ceil(counts[k][l])/counts_per_bin)])
                
                lst=[np.nan]*(len(abund)-len(xvals)) 
                lst2=[np.nan]*(len(abund)-2) 
                lst3=['#ffffff']*(len(abund)-len(xvals)) 

                name1='xbins_'+colname1+'_'+colname2
                name2='ybins_'+colname1+'_'+colname2
                name3='count_'+colname1+'_'+colname2
                name4='color_'+colname1+'_'+colname2
                name5='wi_'+colname1+'_'+colname2
                name6='he_'+colname1+'_'+colname2

                abund[name1]=(pd.concat([pd.Series(xvals),pd.Series(lst)], ignore_index=True))
                abund[name2]=(pd.concat([pd.Series(yvals),pd.Series(lst)], ignore_index=True))
                abund[name3]=(pd.concat([pd.Series(cvals),pd.Series(lst)], ignore_index=True))
                abund[name4]=(pd.concat([pd.Series(colvals),pd.Series(lst3)], ignore_index=True))
                abund[name5]=wi
                abund[name6]=he


abund['x']=abund[x_name]
abund['y']=abund[y_name] 
abund['hhist']=abund['hh_feh_cannon']
abund['hedges_right']=abund['her_feh_cannon'] 
abund['hedges_left']=abund['hel_feh_cannon']
abund['vhist']=abund['hh_alpha_fe_sme']
abund['vedges_right']=abund['her_alpha_fe_sme']
abund['vedges_left']=abund['hel_alpha_fe_sme']
abund['start_x']=np.nan
abund['end_x']=np.nan
abund['start_y']=np.nan
abund['end_y']=np.nan
abund.loc[0,'start_x']=np.min(abund['x'])
abund.loc[0,'end_x']=np.max(abund['x'])
abund.loc[0,'start_y']=np.min(abund['y'])
abund.loc[0,'end_y']=np.max(abund['y'])     
abund['xbins']=abund['xbins_feh_cannon_alpha_fe_sme']
abund['ybins']=abund['ybins_feh_cannon_alpha_fe_sme']
abund['counts']=abund['count_feh_cannon_alpha_fe_sme']
abund['color']=abund['color_feh_cannon_alpha_fe_sme']
abund['wi']=abund['wi_feh_cannon_alpha_fe_sme']
abund['he']=abund['he_feh_cannon_alpha_fe_sme']
abund['ynamer']=''
abund.loc[0,'ynamer']=y_name
abund['xnamer']=''
abund.loc[0,'xnamer']=x_name


  
source = ColumnDataSource(data=dict(abund))

callback = CustomJS(args=dict(source=source), code="""
    
    axis_map = {
            "[Fe/H] Cannon": "feh_cannon",
            "[a/Fe] Cannon": "alpha_fe_cannon",
            "Teff Cannon" : "teff_cannon",
            "logg Cannon" : "logg_cannon",
            "[Fe/H] SME" : "feh_sme",
            "[a/Fe] SME": "alpha_fe_sme",
            "Teff SME" : "teff_sme",
            "logg SME" : "logg_sme",
            "RA" : "ra",
            "Dec" : "dec",
            "RV Cannon": "rv_cannon",
            "RV SME" : "rv_sme",
            "RV GUESS" : "rv_guess",
            "[Fe/H] GUESS": "feh_guess",
            "Teff GUESS" : "teff_guess",
            "logg GUESS" : "logg_guess",
            "RV GUESS" : "rv_guess",
            "Li": "li_abund_sme",
            "C" : "c_abund_sme",
            "O" : "o_abund_sme",
            "Na" : "na_abund_sme",
            "Mg" : "mg_abund_sme", 
            "Al" : "al_abund_sme", 
            "Si" : "si_abund_sme", 
            "K"  : "k_abund_sme",
            "Ca" : "ca_abund_sme", 
            "Sc" : "sc_abund_sme",
            "Ti" : "ti_abund_sme", 
            "V"  : "v_abund_sme",
            "Cr" : "cr_abund_sme", 
            "Mn" : "mn_abund_sme",
            "Co" : "co_abund_sme",
            "Ni" : "ni_abund_sme",
            "Cu" : "cu_abund_sme",
            "Zn" : "zn_abund_sme",
            "Y"  : "y_abund_sme",
            "Ba" : "ba_abund_sme",
            "La" : "la_abund_sme",
            "Nd" : "nd_abund_sme",
            "Eu" : "eu_abund_sme",
            "Rb" : "rb_abund_sme"
        }

    var data = source.get('data');
    var f = cb_obj.get('value')
    x_name=axis_map[f]
    temp = data[x_name]
    
    y_name=data['ynamer'][0]
    data['xnamer'][0]=x_name

    x = data['x']
    y = data['y']
    for (i = 0; i < temp.length; i++) {
        x[i] = Math.pow(temp[i], 1)
    }
    start_x = data['start_x']
    end_x = data['end_x']
    start_x[0] = x.min_value
    end_x[0] = x.max_value

    hhist=data['hhist']
    hedges_left=data['hedges_left']
    hedges_right=data['hedges_right']
    name2='hh_'+x_name
    name3='hel_'+x_name
    name4='her_'+x_name
    temp2 = data[name2]
    temp3 = data[name3]
    temp4 = data[name4]
    hhist=data['hhist']
    hedges_left=data['hedges_left']
    hedges_right=data['hedges_right']
    
    for (i=0; i<x.length; i++) { 
        hhist[i]=Math.pow(temp2[i],1)
        hedges_left[i]=Math.pow(temp3[i],1)
        hedges_right[i]=Math.pow(temp4[i],1)
    }
     

    xbins=data['xbins']
    ybins=data['ybins']
    counts=data['counts']
    color=data['color']
    wi=data['wi']
    he=data['he']

    name5='xbins_'+x_name+'_'+y_name
    name6='ybins_'+x_name+'_'+y_name
    name7='count_'+x_name+'_'+y_name
    name8='color_'+x_name+'_'+y_name
    name9='wi_'+x_name+'_'+y_name
    name10='he_'+x_name+'_'+y_name
    
    temp5=data[name5]
    temp6=data[name6]
    temp7=data[name7]
    temp8=data[name8]
    temp9=data[name9]
    temp10=data[name10]

    for (i=0; i<x.length;i++) {
        xbins[i]=Math.pow(temp5[i],1)
        ybins[i]=Math.pow(temp6[i],1)
        counts[i]=Math.pow(temp7[i],1)
        color[i]=temp8[i]
        wi[i]=temp9[i]
        he[i]=temp10[i]
    }
    
     
    source.trigger('change');
""")

callback2 = CustomJS(args=dict(source=source), code="""
    axis_map = {
            "[Fe/H] Cannon": "feh_cannon",
            "[a/Fe] Cannon": "alpha_fe_cannon",
            "Teff Cannon" : "teff_cannon",
            "logg Cannon" : "logg_cannon",
            "[Fe/H] SME" : "feh_sme",
            "[a/Fe] SME": "alpha_fe_sme",
            "Teff SME" : "teff_sme",
            "logg SME" : "logg_sme",
            "RA" : "ra",
            "Dec" : "dec",
            "RV Cannon": "rv_cannon",
            "RV SME" : "rv_sme",
            "RV GUESS" : "rv_guess",
            "[Fe/H] GUESS": "feh_guess",
            "Teff GUESS" : "teff_guess",
            "logg GUESS" : "logg_guess",
            "RV GUESS" : "rv_guess",
            "Li": "li_abund_sme",
            "C" : "c_abund_sme",
            "O" : "o_abund_sme",
            "Na" : "na_abund_sme",
            "Mg" : "mg_abund_sme", 
            "Al" : "al_abund_sme", 
            "Si" : "si_abund_sme", 
            "K"  : "k_abund_sme",
            "Ca" : "ca_abund_sme", 
            "Sc" : "sc_abund_sme",
            "Ti" : "ti_abund_sme", 
            "V"  : "v_abund_sme",
            "Cr" : "cr_abund_sme", 
            "Mn" : "mn_abund_sme",
            "Co" : "co_abund_sme",
            "Ni" : "ni_abund_sme",
            "Cu" : "cu_abund_sme",
            "Zn" : "zn_abund_sme",
            "Y"  : "y_abund_sme",
            "Ba" : "ba_abund_sme",
            "La" : "la_abund_sme",
            "Nd" : "nd_abund_sme",
            "Eu" : "eu_abund_sme",
            "Rb" : "rb_abund_sme"
        }

    var data = source.get('data');
    var f = cb_obj.get('value')
    y_name=axis_map[f]
    temp = data[y_name]

    x_name=data['xnamer'][0]
    data['ynamer'][0]=y_name

    x = data['x']
    y = data['y']
    for (i = 0; i < temp.length; i++) {
        y[i] = Math.pow(temp[i], 1)
    }
    start_x = data['start_x']
    end_x = data['end_x']
    start_x[0] = x.min_value
    end_x[0] = x.max_value
 
    vhist=data['vhist']
    vedges_left=data['vedges_left']
    vedges_right=data['vedges_right']
    name2='hh_'+y_name
    name3='hel_'+y_name
    name4='her_'+y_name
    temp2 = data[name2]
    temp3 = data[name3]
    temp4 = data[name4]
    
    for (i=0; i<x.length; i++) { 
        vhist[i]=Math.pow(temp2[i],1)
        vedges_left[i]=Math.pow(temp3[i],1)
        vedges_right[i]=Math.pow(temp4[i],1)
    }
 
    xbins=data['xbins']
    ybins=data['ybins']
    counts=data['counts']
    color=data['color']
    wi=data['wi']
    he=data['he']

    name5='xbins_'+x_name+'_'+y_name
    name6='ybins_'+x_name+'_'+y_name
    name7='count_'+x_name+'_'+y_name
    name8='color_'+x_name+'_'+y_name
    name9='wi_'+x_name+'_'+y_name
    name10='he_'+x_name+'_'+y_name
    
    temp5=data[name5]
    temp6=data[name6]
    temp7=data[name7]
    temp8=data[name8]
    temp9=data[name9]
    temp10=data[name10]

    for (i=0; i<x.length;i++) {
        xbins[i]=Math.pow(temp5[i],1)
        ybins[i]=Math.pow(temp6[i],1)
        counts[i]=Math.pow(temp7[i],1)
        color[i]=temp8[i]
        wi[i]=temp9[i]
        he[i]=temp10[i]
    }

    source.trigger('change');
""")

callback3 = CustomJS(args=dict(source=source), code="""
        var data = source.get('data');
        x = data['x']
        y = data['y']
        start_x = data['start_x']
        end_x = data['end_x']
        
        var f = cb_obj.get('value')
        
        if (f == "X axis"){
            start_x[0] = x.max_value;
            end_x[0] = x.min_value;           
        }
        source.trigger('change');
""")

select_x=Select(title="X axis:", value="[Fe/H] Cannon", options=list(np.sort(axis_map.keys())), callback=callback)
select_y=Select(title="Y axis:", value="[a/Fe] SME", options=list(np.sort(axis_map.keys())), callback=callback2)
#axes_select=Select(title="Flip:", value='None', options=['None', 'X axis', 'Y axis', 'Both'], callback=callback3)

#xr = Range1d(start=source.data['start_x'][0],end=source.data['end_x'][0])
#yr = Range1d(start=min(abund['y']),end=max(abund['y']))

## Make Scatter plot for first tab
scatter_plot = Figure(tools=TOOLS, plot_width=600, plot_height=600, title=None, min_border=10, \
                      min_border_left=50)
scatter_plot.scatter('x','y', size=3, source=source, color="#3A5785", alpha=0.6)
tab1 = Panel(child=scatter_plot, title="Scatter")

## Create the Density plot for the second tab
density_plot = Figure(tools=TOOLS2,plot_width=600, plot_height=600, title=None, min_border=10,\
                      min_border_left=50,x_range=scatter_plot.x_range,y_range=scatter_plot.y_range)

density_plot.rect('xbins', 'ybins', width='wi', height='he', color='color', source=source)
density_plot.xgrid.grid_line_color = None
density_plot.ygrid.grid_line_color = None
tab2 = Panel(child=density_plot, title="Number Density")

tabs = Tabs(tabs=[ tab1, tab2 ])

## create the horizontal histogram
LINE_ARGS = dict(color="#3A5785", line_color=None)
ph = Figure(toolbar_location=None, plot_width=scatter_plot.plot_width-37, plot_height=200, x_range=scatter_plot.x_range,
                title=None, min_border=10, min_border_left=50)
ph.xgrid.grid_line_color = None
ph.quad(bottom=0, left='hedges_left', right='hedges_right', top='hhist', color="white", line_color="#3A5785", source=source)
ph.min_border_top = 10
ph.min_border_right = 10

## create the vertical histogram
th = 42 ## need to adjust for toolbar height
pv = Figure(toolbar_location=None, plot_width=200, plot_height=scatter_plot.plot_height+th+45,
                y_range=scatter_plot.y_range, title=None, min_border=10, min_border_top=th)
pv.ygrid.grid_line_color = None
pv.xaxis.major_label_orientation = -3.14/2
pv.quad(left=0, bottom='vedges_left', top='vedges_right', right='vhist', color="white", line_color="#3A5785", source=source)
pv.min_border_top = 140
pv.min_border_left = 0
 
layout=vform(hplot(select_x, select_y), vplot(hplot(tabs, pv), hplot(ph, Paragraph(width=200))), width=800, height=800)
 
show(layout)

In [32]:
from bokeh.io import vform
from bokeh.models import CustomJS, ColumnDataSource, Select
from bokeh.plotting import Figure, output_file, show, hplot, vplot
from bokeh.models import ColumnDataSource, HBox, VBoxForm,Range1d, HoverTool, BoxZoomTool, \
ResetTool, ResizeTool, PreviewSaveTool, PanTool, BoxSelectTool, LassoSelectTool, Paragraph, CustomJS
from bokeh.models.widgets import CheckboxGroup, Panel, Tabs
from astropy.table import Table
import numpy as np
import pandas as pd
from bokeh.models import Range1d, Slider
from bokeh.palettes import RdPu9
import matplotlib.pyplot as plt



TOOLS= [BoxZoomTool(), ResetTool(), ResizeTool(), PanTool(), HoverTool(tooltips=[("sobject_id","@sobject_id")])]
TOOLS2= [BoxZoomTool(),ResetTool(), ResizeTool(), PanTool(), HoverTool(tooltips=[("counts","@counts")])]

abund=Table.read('/Users/kschles/Documents/GALAH/wg4output/wg4_04292016/sobject_iraf_k2.fits', format='fits')
test=Table.to_pandas(abund)
abund_orig=test
test=abund_orig.loc[0:5000]
histcolumns=['sobject_id', 'ra', 'dec', 'teff_cannon', 'logg_cannon', 'feh_cannon', 'alpha_fe_cannon',\
             'alpha_fe_sme', 'teff_guess', 'logg_guess',\
             'feh_guess', 'rv_guess', 'rv_cannon','rv_sme', 'mg_abund_sme','si_abund_sme',\
             'ca_abund_sme','ti_abund_sme']

abund=test.loc[:,histcolumns]

output_file("callback.html")

axis_map = {
        "[Fe/H] Cannon": "feh_cannon",
        "[a/Fe] Cannon": "alpha_fe_cannon",
        "Teff Cannon" : "teff_cannon",
        "logg Cannon" : "logg_cannon",
        #"[Fe/H] SME" : "feh_sme",
        "[a/Fe] SME": "alpha_fe_sme",
        #"Teff SME" : "teff_sme",
        #"logg SME" : "logg_sme",
        "RA" : "ra",
        "Dec" : "dec",
        "RV Cannon": "rv_cannon",
        "RV SME" : "rv_sme",
        "RV GUESS" : "rv_guess",
        "[Fe/H] GUESS": "feh_guess",
        "Teff GUESS" : "teff_guess",
        "logg GUESS" : "logg_guess",
        "RV GUESS" : "rv_guess",
        #"Li": "li_abund_sme",
        #"C" : "c_abund_sme",
        #"O" : "o_abund_sme",
        #"Na" : "na_abund_sme",
        "Mg" : "mg_abund_sme", 
        #"Al" : "al_abund_sme", 
        "Si" : "si_abund_sme", 
        #"K"  : "k_abund_sme",
        "Ca" : "ca_abund_sme", 
        #"Sc" : "sc_abund_sme",
        "Ti" : "ti_abund_sme", 
        #"V"  : "v_abund_sme",
        #"Cr" : "cr_abund_sme", 
        #"Mn" : "mn_abund_sme",
        #"Co" : "co_abund_sme",
        #"Ni" : "ni_abund_sme",
        #"Cu" : "cu_abund_sme",
        #"Zn" : "zn_abund_sme",
        #"Y"  : "y_abund_sme",
        #"Ba" : "ba_abund_sme",
        #"La" : "la_abund_sme",
        #"Nd" : "nd_abund_sme",
        #"Eu" : "eu_abund_sme",
        #"Rb" : "rb_abund_sme"
        }


xname="[Fe/H] Cannon"
yname="[a/Fe] SME"
binval='50'

## Set up initial data: 
x_name = axis_map[xname]
y_name = axis_map[yname]

xbinnum=int(binval)
ybinnum=int(binval)


## Loop through columns in abund to add histograms to dictionary for source call in bokeh interactive.
for i in range(1,len(histcolumns)): 
    colname=histcolumns[i]
    hhist, hedges=np.histogram(abund.loc[np.where(np.isnan(abund[colname])==False)[0],colname], bins=xbinnum)
    lst=[np.nan]*(len(abund)-xbinnum) 
    lst2=[np.nan]*(len(abund)-(xbinnum+1)) 
    name1='hh_'+colname
    name2='hel_'+colname
    name3='her_'+colname
    abund[name1]=(pd.concat([pd.Series(hhist),pd.Series(lst)], ignore_index=True))
    abund[name2]=(pd.concat([pd.Series(hedges[:-1]),pd.Series(lst2)], ignore_index=True))
    abund[name3]=(pd.concat([pd.Series(hedges[1:]),pd.Series(lst2)], ignore_index=True))

## Loop through columns to add density distributions to dictionary for source call. 

histcolumns2=['sobject_id', 'ra', 'dec', 'teff_cannon', 'logg_cannon', 'feh_cannon', 'alpha_fe_cannon',\
             'alpha_fe_sme', 'teff_guess', 'logg_guess',\
             'feh_guess', 'rv_guess', 'rv_cannon','rv_sme', 'mg_abund_sme','si_abund_sme',\
             'ca_abund_sme','ti_abund_sme']

for f in range(1,len(histcolumns2)): 
    colname1=histcolumns2[f]
    for g in range(1,len(histcolumns2)): 
        colname2=histcolumns2[g]
        if (colname2!=colname1): 
            subset=abund[[colname1,colname2]].dropna()
            subset.reset_index(drop=True, inplace=True)
            if (len(subset)>50): 
                counts, xb, yb, image=plt.hist2d(subset[colname1],subset[colname2],bins=50)
                wi=xb[1]-xb[0]
                he=yb[1]-yb[0]
                colors=['#ffffff']+RdPu9[::-1]
                
                xvals=[]
                yvals=[]
                cvals=[]
                colvals=[]
                if (np.max(counts)!=np.min(counts)): 
                    counts_per_bin=(np.max(counts)-np.min(counts))/8.
                else :
                    print 'Counts Error: ', colname1,colname2
                
                for k in range(len(xb)-1): 
                    for l in range(len(xb)-1): 
                        xvals.append(xb[k])
                        yvals.append(yb[l])
                        cvals.append(counts[k][l])
                        colvals.append(colors[int(np.ceil(counts[k][l])/counts_per_bin)])
                
                lst=[np.nan]*(len(abund)-len(xvals)) 
                lst2=[np.nan]*(len(abund)-2) 
                lst3=['#ffffff']*(len(abund)-len(xvals)) 

                name1='xbins_'+colname1+'_'+colname2
                name2='ybins_'+colname1+'_'+colname2
                name3='count_'+colname1+'_'+colname2
                name4='color_'+colname1+'_'+colname2
                name5='wi_'+colname1+'_'+colname2
                name6='he_'+colname1+'_'+colname2

                abund[name1]=(pd.concat([pd.Series(xvals),pd.Series(lst)], ignore_index=True))
                abund[name2]=(pd.concat([pd.Series(yvals),pd.Series(lst)], ignore_index=True))
                abund[name3]=(pd.concat([pd.Series(cvals),pd.Series(lst)], ignore_index=True))
                abund[name4]=(pd.concat([pd.Series(colvals),pd.Series(lst3)], ignore_index=True))
                abund[name5]=wi
                abund[name6]=he


abund['x']=abund[x_name]
abund['y']=abund[y_name] 
abund['hhist']=abund['hh_feh_cannon']
abund['hedges_right']=abund['her_feh_cannon'] 
abund['hedges_left']=abund['hel_feh_cannon']
abund['vhist']=abund['hh_alpha_fe_sme']
abund['vedges_right']=abund['her_alpha_fe_sme']
abund['vedges_left']=abund['hel_alpha_fe_sme']
abund['start_x']=np.nan
abund['end_x']=np.nan
abund['start_y']=np.nan
abund['end_y']=np.nan
abund.loc[0,'start_x']=np.min(abund['x'])
abund.loc[0,'end_x']=np.max(abund['x'])
abund.loc[0,'start_y']=np.min(abund['y'])
abund.loc[0,'end_y']=np.max(abund['y'])     
abund['xbins']=abund['xbins_feh_cannon_alpha_fe_sme']
abund['ybins']=abund['ybins_feh_cannon_alpha_fe_sme']
abund['counts']=abund['count_feh_cannon_alpha_fe_sme']
abund['color']=abund['color_feh_cannon_alpha_fe_sme']
abund['wi']=abund['wi_feh_cannon_alpha_fe_sme']
abund['he']=abund['he_feh_cannon_alpha_fe_sme']
abund['ynamer']=''
abund.loc[0,'ynamer']=y_name
abund['xnamer']=''
abund.loc[0,'xnamer']=x_name
abund['oneone_x']=np.sort(abund['x'])
abund['oneone_y']=abund['oneone_x']

source = ColumnDataSource(data=dict(abund))

callback = CustomJS(args=dict(source=source), code="""
    
    axis_map = {
            "[Fe/H] Cannon": "feh_cannon",
            "[a/Fe] Cannon": "alpha_fe_cannon",
            "Teff Cannon" : "teff_cannon",
            "logg Cannon" : "logg_cannon",
            "[Fe/H] SME" : "feh_sme",
            "[a/Fe] SME": "alpha_fe_sme",
            "Teff SME" : "teff_sme",
            "logg SME" : "logg_sme",
            "RA" : "ra",
            "Dec" : "dec",
            "RV Cannon": "rv_cannon",
            "RV SME" : "rv_sme",
            "RV GUESS" : "rv_guess",
            "[Fe/H] GUESS": "feh_guess",
            "Teff GUESS" : "teff_guess",
            "logg GUESS" : "logg_guess",
            "RV GUESS" : "rv_guess",
            "Li": "li_abund_sme",
            "C" : "c_abund_sme",
            "O" : "o_abund_sme",
            "Na" : "na_abund_sme",
            "Mg" : "mg_abund_sme", 
            "Al" : "al_abund_sme", 
            "Si" : "si_abund_sme", 
            "K"  : "k_abund_sme",
            "Ca" : "ca_abund_sme", 
            "Sc" : "sc_abund_sme",
            "Ti" : "ti_abund_sme", 
            "V"  : "v_abund_sme",
            "Cr" : "cr_abund_sme", 
            "Mn" : "mn_abund_sme",
            "Co" : "co_abund_sme",
            "Ni" : "ni_abund_sme",
            "Cu" : "cu_abund_sme",
            "Zn" : "zn_abund_sme",
            "Y"  : "y_abund_sme",
            "Ba" : "ba_abund_sme",
            "La" : "la_abund_sme",
            "Nd" : "nd_abund_sme",
            "Eu" : "eu_abund_sme",
            "Rb" : "rb_abund_sme"
        }

    var data = source.get('data');
    var f = cb_obj.get('value')
    x_name=axis_map[f]
    temp = data[x_name]
    
    y_name=data['ynamer'][0]
    data['xnamer'][0]=x_name

    x = data['x']
    y = data['y']
    for (i = 0; i < temp.length; i++) {
        x[i] = Math.pow(temp[i], 1)
    }
    start_x = data['start_x']
    end_x = data['end_x']
    start_x[0] = x.min_value
    end_x[0] = x.max_value

    hhist=data['hhist']
    hedges_left=data['hedges_left']
    hedges_right=data['hedges_right']
    name2='hh_'+x_name
    name3='hel_'+x_name
    name4='her_'+x_name
    temp2 = data[name2]
    temp3 = data[name3]
    temp4 = data[name4]
    hhist=data['hhist']
    hedges_left=data['hedges_left']
    hedges_right=data['hedges_right']
    
    for (i=0; i<x.length; i++) { 
        hhist[i]=Math.pow(temp2[i],1)
        hedges_left[i]=Math.pow(temp3[i],1)
        hedges_right[i]=Math.pow(temp4[i],1)
    }
     

    xbins=data['xbins']
    ybins=data['ybins']
    counts=data['counts']
    color=data['color']
    wi=data['wi']
    he=data['he']

    name5='xbins_'+x_name+'_'+y_name
    name6='ybins_'+x_name+'_'+y_name
    name7='count_'+x_name+'_'+y_name
    name8='color_'+x_name+'_'+y_name
    name9='wi_'+x_name+'_'+y_name
    name10='he_'+x_name+'_'+y_name
    
    temp5=data[name5]
    temp6=data[name6]
    temp7=data[name7]
    temp8=data[name8]
    temp9=data[name9]
    temp10=data[name10]

    for (i=0; i<x.length;i++) {
        xbins[i]=Math.pow(temp5[i],1)
        ybins[i]=Math.pow(temp6[i],1)
        counts[i]=Math.pow(temp7[i],1)
        color[i]=temp8[i]
        wi[i]=temp9[i]
        he[i]=temp10[i]
    }
    
    oneone_x=data['oneone_x']
    oneone_y=data['oneone_y']
    
    for (i=0;i<x.length;i++) {
        oneone_x[i]=x.sort()[i]
        oneone_y[i]=Math.pow(oneone_x[i],1)    
    }
     
    source.trigger('change');
""")

callback2 = CustomJS(args=dict(source=source), code="""
    axis_map = {
            "[Fe/H] Cannon": "feh_cannon",
            "[a/Fe] Cannon": "alpha_fe_cannon",
            "Teff Cannon" : "teff_cannon",
            "logg Cannon" : "logg_cannon",
            "[Fe/H] SME" : "feh_sme",
            "[a/Fe] SME": "alpha_fe_sme",
            "Teff SME" : "teff_sme",
            "logg SME" : "logg_sme",
            "RA" : "ra",
            "Dec" : "dec",
            "RV Cannon": "rv_cannon",
            "RV SME" : "rv_sme",
            "RV GUESS" : "rv_guess",
            "[Fe/H] GUESS": "feh_guess",
            "Teff GUESS" : "teff_guess",
            "logg GUESS" : "logg_guess",
            "RV GUESS" : "rv_guess",
            "Li": "li_abund_sme",
            "C" : "c_abund_sme",
            "O" : "o_abund_sme",
            "Na" : "na_abund_sme",
            "Mg" : "mg_abund_sme", 
            "Al" : "al_abund_sme", 
            "Si" : "si_abund_sme", 
            "K"  : "k_abund_sme",
            "Ca" : "ca_abund_sme", 
            "Sc" : "sc_abund_sme",
            "Ti" : "ti_abund_sme", 
            "V"  : "v_abund_sme",
            "Cr" : "cr_abund_sme", 
            "Mn" : "mn_abund_sme",
            "Co" : "co_abund_sme",
            "Ni" : "ni_abund_sme",
            "Cu" : "cu_abund_sme",
            "Zn" : "zn_abund_sme",
            "Y"  : "y_abund_sme",
            "Ba" : "ba_abund_sme",
            "La" : "la_abund_sme",
            "Nd" : "nd_abund_sme",
            "Eu" : "eu_abund_sme",
            "Rb" : "rb_abund_sme"
        }

    var data = source.get('data');
    var f = cb_obj.get('value')
    y_name=axis_map[f]
    temp = data[y_name]

    x_name=data['xnamer'][0]
    data['ynamer'][0]=y_name

    x = data['x']
    y = data['y']
    for (i = 0; i < temp.length; i++) {
        y[i] = Math.pow(temp[i], 1)
    }
    start_x = data['start_x']
    end_x = data['end_x']
    start_x[0] = x.min_value
    end_x[0] = x.max_value
 
    vhist=data['vhist']
    vedges_left=data['vedges_left']
    vedges_right=data['vedges_right']
    name2='hh_'+y_name
    name3='hel_'+y_name
    name4='her_'+y_name
    temp2 = data[name2]
    temp3 = data[name3]
    temp4 = data[name4]
    
    for (i=0; i<x.length; i++) { 
        vhist[i]=Math.pow(temp2[i],1)
        vedges_left[i]=Math.pow(temp3[i],1)
        vedges_right[i]=Math.pow(temp4[i],1)
    }
 
    xbins=data['xbins']
    ybins=data['ybins']
    counts=data['counts']
    color=data['color']
    wi=data['wi']
    he=data['he']

    name5='xbins_'+x_name+'_'+y_name
    name6='ybins_'+x_name+'_'+y_name
    name7='count_'+x_name+'_'+y_name
    name8='color_'+x_name+'_'+y_name
    name9='wi_'+x_name+'_'+y_name
    name10='he_'+x_name+'_'+y_name
    
    temp5=data[name5]
    temp6=data[name6]
    temp7=data[name7]
    temp8=data[name8]
    temp9=data[name9]
    temp10=data[name10]

    for (i=0; i<x.length;i++) {
        xbins[i]=Math.pow(temp5[i],1)
        ybins[i]=Math.pow(temp6[i],1)
        counts[i]=Math.pow(temp7[i],1)
        color[i]=temp8[i]
        wi[i]=temp9[i]
        he[i]=temp10[i]
    }

    oneone_x=data['oneone_x']
    oneone_y=data['oneone_y']
    
    for (i=0;i<x.length;i++) {
        oneone_x[i]=x.sort()[i]
        oneone_y[i]=Math.pow(oneone_x[i],1)    
    }

    source.trigger('change');
""")

callback4=CustomJS(args=dict(source=source), code="""
    data = source.get('data')
    var k = cb_obj.get('value')
    oneone_x, oneone_y = data['oneone_x'], data['oneone_y']
    for (i=0; i<oneone_x.length;i++) {
        oneone_x[i] = Math.pow(oneone_x[i], 1)
        oneone_y[i] = Math.pow(oneone_x[i], k)
    }
    source.trigger('change');
""")

select_x=Select(title="X axis:", value="[Fe/H] Cannon", options=list(np.sort(axis_map.keys())), callback=callback)
select_y=Select(title="Y axis:", value="[a/Fe] SME", options=list(np.sort(axis_map.keys())), callback=callback2)
slider = Slider(start=0.1, end=4, value=1, step=.1, title="power", callback=callback4)

#oneone_check=CheckboxGroup(labels=["Overplot One-to-One"], active=[], callback4)
#axes_select=Select(title="Flip:", value='None', options=['None', 'X axis', 'Y axis', 'Both'], callback=callback3)

#xr = Range1d(start=source.data['start_x'][0],end=source.data['end_x'][0])
#yr = Range1d(start=min(abund['y']),end=max(abund['y']))

## Make Scatter plot for first tab
scatter_plot = Figure(tools=TOOLS, plot_width=600, plot_height=600, title=None, min_border=10, \
                      min_border_left=50)
scatter_plot.scatter('x','y', size=3, source=source, color="#3A5785", alpha=0.6)
scatter_plot.line('oneone_x', 'oneone_y', source=source, alpha=0.2, color="red")
tab1 = Panel(child=scatter_plot, title="Scatter")


## Create the Density plot for the second tab
density_plot = Figure(tools=TOOLS2,plot_width=600, plot_height=600, title=None, min_border=10,\
                      min_border_left=50,x_range=scatter_plot.x_range,y_range=scatter_plot.y_range)

density_plot.rect('xbins', 'ybins', width='wi', height='he', color='color', source=source)
density_plot.line('oneone_x', 'oneone_y', source=source, alpha=0.2, color="red")
density_plot.xgrid.grid_line_color = None
density_plot.ygrid.grid_line_color = None
tab2 = Panel(child=density_plot, title="Number Density")

tabs = Tabs(tabs=[ tab1, tab2 ])

## create the horizontal histogram
LINE_ARGS = dict(color="#3A5785", line_color=None)
ph = Figure(toolbar_location=None, plot_width=scatter_plot.plot_width-37, plot_height=200, x_range=scatter_plot.x_range,
                title=None, min_border=10, min_border_left=50)
ph.xgrid.grid_line_color = None
ph.quad(bottom=0, left='hedges_left' , right='hedges_right', top='hhist', color="white", line_color="#3A5785", source=source)
ph.min_border_top = 10
ph.min_border_right = 10

## create the vertical histogram
th = 42 ## need to adjust for toolbar height
pv = Figure(toolbar_location=None, plot_width=200, plot_height=scatter_plot.plot_height+th+45,
                y_range=scatter_plot.y_range, title=None, min_border=10, min_border_top=th)
pv.ygrid.grid_line_color = None
pv.xaxis.major_label_orientation = -3.14/2
pv.quad(left=0, bottom='vedges_left', top='vedges_right', right='vhist', color="white", line_color="#3A5785", source=source)
pv.min_border_top = 140
pv.min_border_left = 0
 
layout=vform(vplot(hplot(select_x, select_y), slider), vplot(hplot(tabs, pv), hplot(ph, Paragraph(width=200))), width=800, height=800)
 
show(layout)

In [31]:
f

17

In [2]:
from bokeh.embed import components
script, div = components([select_x, select_y, tabs, pv, ph])

div

(u'\n<div class="plotdiv" id="0c25e65a-da3f-457f-8bca-b5e3ac9b2ff2"></div>',
 u'\n<div class="plotdiv" id="2b43a383-8dc8-4405-ae82-1cdef320c68f"></div>',
 u'\n<div class="plotdiv" id="56e8550c-28e1-4d48-95cd-7b3820c088e2"></div>',
 u'\n<div class="plotdiv" id="aee8bf07-edc3-4c8e-a335-c3b528a23d03"></div>',
 u'\n<div class="plotdiv" id="ccb78b7b-d3fe-4dae-a46f-e2d856d1a19f"></div>')

In [4]:
f=open('bokeh3_new.js', 'w')
f.write(script)
f.close()


In [22]:
abund['oneone_x']=np.sort(abund['x'])
abund['oneone_y']=abund['oneone_x']

-1.64602 0.820819


In [9]:
oneone_check=CheckboxGroup(labels=["Overplot One-to-One"], active=[], callback=callback4)

AttributeError: unexpected attribute 'callback' to CheckboxGroup, possible attributes are active, disabled, inline, labels, name or tags

In [8]:
callback4 = CustomJS(args=dict(source=source), code="""
        var data = source.get('data');
        x = data['x']
        y = data['y']
        
        oneone_x=data['oneone_x']
        oneone_y=data['oneone_y']
        
        for (i=0; i<oneone_x.length;i++) {
            oneone_x[i]=Math.pow(oneone_x[i],1)
            oneone_y[i]=Math.pow(oneone_x[i],1)
        }

         source.trigger('change');
""")